In [1]:
import abc
import copy
import functools
import itertools
import math
import random
from typing import Any, Optional, Sequence


In [2]:
import gym_electric_motor as gem
import gymnasium as gym
import numpy as np
from gym_electric_motor import reward_functions
from scipy import linalg
from sklearn import mixture


In [3]:
from ray import tune

In [4]:
from rlplg.environments import gridworld, redgreen

## Tiling

In [6]:
"""
Tile Coding Software version 3.0beta
by Rich Sutton
based on a program created by Steph Schaeffer and others
External documentation and recommendations on the use of this code is available in the 
reinforcement learning textbook by Sutton and Barto, and on the web.
These need to be understood before this code is.

This software is for Python 3 or more.

This is an implementation of grid-style tile codings, based originally on
the UNH CMAC code (see http://www.ece.unh.edu/robots/cmac.htm), but by now highly changed. 
Here we provide a function, "tiles", that maps floating and integer
variables to a list of tiles, and a second function "tiles-wrap" that does the same while
wrapping some floats to provided widths (the lower wrap value is always 0).

The float variables will be gridded at unit intervals, so generalization
will be by approximately 1 in each direction, and any scaling will have 
to be done externally before calling tiles.

Num-tilings should be a power of 2, e.g., 16. To make the offsetting work properly, it should
also be greater than or equal to four times the number of floats.

The first argument is either an index hash table of a given size (created by (make-iht size)), 
an integer "size" (range of the indices from 0), or nil (for testing, indicating that the tile 
coordinates are to be returned without being converted to indices).
"""

basehash = hash

class IHT:
    "Structure to handle collisions"
    def __init__(self, sizeval):
        self.size = sizeval                        
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        "Prepares a string for printing whenever this object is printed"
        return "Collision table:" + \
               " size:" + str(self.size) + \
               " overfullCount:" + str(self.overfullCount) + \
               " dictionary:" + str(len(self.dictionary)) + " items"

    def count(self):
        return len(self.dictionary)
    
    def fullp(self):
        return len(self.dictionary) >= self.size
    
    def getindex(self, obj, readonly=False):
        d = self.dictionary
        if obj in d:
            return d[obj]
        elif readonly:
            return None
        size = self.size
        count = self.count()
        if count >= size:
            # TODO: Fail
            if self.overfullCount==0: 
                print('IHT full, starting to allow collisions')
            self.overfullCount += 1
            return basehash(obj) % self.size
        else:
            d[obj] = count
            return count

def hashcoords(coordinates, m, readonly=False):
    if isinstance(m, IHT):
        return m.getindex(tuple(coordinates), readonly)
    if isinstance(m, int):
        return basehash(tuple(coordinates)) % m
    if m is None:
        return coordinates


def tiles(ihtORsize, numtilings, floats, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    qfloats = [math.floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append( (q + b) // numtilings )
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles

def tileswrap (ihtORsize, numtilings, floats, wrapwidths, ints=[], readonly=False):
    """returns num-tilings tile indices corresponding to the floats and ints, wrapping some floats"""
    qfloats = [math.floor(f*numtilings) for f in floats]
    Tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling*2
        coords = [tiling]
        b = tiling
        for q, width in itertools.zip_longest(qfloats, wrapwidths):
            c = (q + b%numtilings) // numtilings
            coords.append(c%width if width else c)
            b += tilingX2
        coords.extend(ints)
        Tiles.append(hashcoords(coords, ihtORsize, readonly))
    return Tiles


In [7]:
def pow2geq(lb):
    exp = 1
    while True:
        rs = np.power(2, exp)
        if rs >= lb:
            break
        exp += 1
    return rs

In [8]:
def solve_least_squares(
    matrix: np.ndarray, rhs: np.ndarray
) -> np.ndarray:
    try:
        solution, _, _, _ = linalg.lstsq(a=matrix, b=rhs, lapack_driver="gelsy")
        return solution  # type: ignore
    except linalg.LinAlgError as err:
        # the computation failed, likely due to the matix being unsuitable (no solution).
        raise ValueError("Failed to solve linear system") from err

In [9]:
def rmse(v_pred: np.ndarray, v_true: np.ndarray, axis: int):
    if np.shape(v_pred) != np.shape(v_true):
        raise ValueError(
            f"Tensors have different shapes: {np.shape(v_pred)} != {np.shape(v_true)}"
        )
    return np.sqrt(
        np.sum(np.power(v_pred - v_true, 2.0), axis=axis) / np.shape(v_pred)[axis]
    )

In [10]:
class Tiles:
    def __init__(
        self, 
        obs_min: np.ndarray, 
        obs_max: np.ndarray,
        num_actions: Sequence[int],
        tiling_dim: int, num_tilings: Optional[int] = None
    ):
        assert isinstance(obs_min, np.ndarray)
        assert isinstance(obs_max, np.ndarray)
        self.obs_min = obs_min
        self.obs_max = obs_max
        self.tiling_dim = tiling_dim
        self.wrapwidths = [tiling_dim] * np.size(obs_min)
        self.num_actions = num_actions
    
        # num tilings should a power of 2
        # and at least 4 times greater than
        # the number of dimensions
        self.num_tilings = num_tilings or pow2geq(np.size(obs_min) * 4)
        self.max_size = (tiling_dim ** np.size(obs_min)) * self.num_tilings
        print("Num tilings", self.num_tilings, "\n", "Flat dim:", self.max_size)
        self.iht = IHT(self.max_size)
        self.dim = self.max_size + self.num_actions

    def __call__(self, obs, action):
        obs_scaled_01 = (obs - self.obs_min) / (self.obs_max - self.obs_min)
        repr = np.zeros(shape=self.dim)
        idx = tileswrap(
            self.iht, 
            numtilings=self.num_tilings, 
            floats=obs_scaled_01 * self.tiling_dim,
            wrapwidths=self.wrapwidths,
            ints=[action] if action else []
        )
        repr[idx] = 1
        print(idx, len(idx))
        return repr


In [11]:
tiles_enc = Tiles(
    np.array([0, 0, 0]),
    np.array([9, 9, 9]),
    num_actions=100,
    tiling_dim=4,
)
tiles_enc(np.array([1, 2, 3]), action=None)
tiles_enc(np.array([1, 2, 3]), action=1)
tiles_enc(np.array([9, 9, 9]), action=90)
tiles_enc(np.array([9, 9, 9]), action=98)
tiles_enc(np.array([9, 9, 8]), action=98)
tiles_enc(np.array([9, 9, 9]), action=99)
tiles_enc(np.array([7, 6, 8]), action=99)

Num tilings 16 
 Flat dim: 1024
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 16
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 16
[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47] 16
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63] 16
[64, 65, 50, 51, 66, 53, 54, 67, 56, 57, 68, 69, 60, 70, 71, 63] 16
[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87] 16
[88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103] 16


array([0., 0., 0., ..., 0., 0., 0.])

## Utility Functions

In [12]:
def collection_traj_data(env, steps: int):
    obs, _ = env.reset()
    step = 0
    buffer = []
    while step < steps:
        action = env.action_space.sample()
        next_obs, rew, term, trunc, _,  = env.step(action)
        step += 1
        buffer.append((obs, action, rew))
        obs = next_obs
        if term or trunc:
            obs, _ = env.reset()
    return buffer

## Feature transformation

In [13]:
def hashtrick(xs, dim: int):
    ys = np.zeros(dim, dtype=np.int32)
    idx,  = np.where(xs == 1)
    for i in idx:
        ys[i % dim] += 1
    return ys

In [14]:
class FeatTransform(abc.ABC):
    def __init__(self):
        pass

    @abc.abstractmethod
    def transform(self, obs: Any, action: Any):
        pass

    @property
    @abc.abstractmethod
    def output_shape(self) -> int:
        pass


### Tile Transform

In [15]:
class TileTransform(FeatTransform):
    def __init__(self, env: gym.Env, tiling_dim: int, num_tilings: int = None, hash_dim: int = None):
        if not isinstance(env.observation_space, gym.spaces.Box):
            raise ValueError("env.observation_space must be `spaces.Box`")
        if not isinstance(env.action_space, gym.spaces.Discrete):
            raise ValueError("env.action_space must be `spaces.Discrete`")            
        
        self.obs_space = env.observation_space
        self.tiling_dim = tiling_dim
        self.wrapwidths = [tiling_dim] * np.size(self.obs_space.low)
        self.num_actions = env.action_space.n
    
        # num tilings should a power of 2
        # and at least 4 times greater than
        # the number of dimensions
        self.num_tilings = num_tilings or pow2geq(np.size(self.obs_space.low) * 4)
        self.max_size = (tiling_dim ** np.size(self.obs_space.low)) * self.num_tilings * self.num_actions
        print("Num tilings", self.num_tilings, "\n", "Flat dim:", self.max_size)
        self.iht = IHT(self.max_size)
        self.hash_dim = hash_dim
        
    def transform(self, obs: Any, action: Any):
        obs_scaled_01 = (obs - self.obs_space.low) / (self.obs_space.high - self.obs_space.low)
        repr = np.zeros(shape=self.max_size)
        idx = tileswrap(
            self.iht, 
            numtilings=self.num_tilings, 
            floats=obs_scaled_01 * self.tiling_dim,
            wrapwidths=self.wrapwidths,
            ints=[action] if action else []
        )
        repr[idx] = 1
        if self.hash_dim:
            return hashtrick(repr, self.hash_dim)
        return repr  

    @property
    def output_shape(self) -> int:
        return self.hash_dim or self.max_size

In [16]:
env = gym.make("MountainCar-v0", max_episode_steps=10000)
transform = TileTransform(env, tiling_dim=8)
obs, _ = env.reset()
print(transform.transform(obs, 0))
print(transform.transform(obs, 1))
print(transform.transform(obs, 2))

Num tilings 8 
 Flat dim: 1536
[1. 1. 1. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


### Scale Transform

In [17]:
class ScaleObsOheActTransform(FeatTransform):
    def __init__(self, env: gym.Env, hash_dim: int = None):
        if not isinstance(env.observation_space, gym.spaces.Box):
            raise ValueError("env.observation_space must be `spaces.Box`")
        if not isinstance(env.action_space, gym.spaces.Discrete):
            raise ValueError("env.action_space must be `spaces.Discrete`")
        
        self.obs_space = env.observation_space
        self.num_actions = env.action_space.n
        self.hash_dim = hash_dim
        self.obs_dim = np.size(self.obs_space.high)

    def transform(self, obs: Any, action: Any):
        obs_scaled_01 = (obs - self.obs_space.low) / (self.obs_space.high - self.obs_space.low)
        output = np.zeros(shape=self.obs_dim * self.num_actions)
        idx = np.size(self.obs_space.high) * action
        output[idx:idx+self.obs_dim] = obs_scaled_01
        return output

    @property
    def output_shape(self) -> int:
        return self.hash_dim or self.obs_dim * self.num_actions

In [18]:
env = gym.make("MountainCar-v0", max_episode_steps=10000)
transform = ScaleObsOheActTransform(env)
obs, _ = env.reset()
print(transform.transform(obs, 0))
print(transform.transform(obs, 1))
print(transform.transform(obs, 2))

[0.33441147 0.5        0.         0.         0.         0.        ]
[0.         0.         0.33441147 0.5        0.         0.        ]
[0.         0.         0.         0.         0.33441147 0.5       ]


### Gaussian Mix Transform

In [19]:
class GaussianMixObsOheActTransform(FeatTransform):
    def __init__(self, env: gym.Env, params, sample_steps: int = 100_000):
        # params or hps_params can be provided
        if not isinstance(env.observation_space, gym.spaces.Box):
            raise ValueError("env.observation_space must be `spaces.Box`")
        if not isinstance(env.action_space, gym.spaces.Discrete):
            raise ValueError("env.action_space must be `spaces.Discrete`")

        self.obs_space = env.observation_space
        self.num_actions = env.action_space.n
        self._gm = mixture.GaussianMixture(**params)
        self._gm.fit(
            [obs for obs, _, _ in collection_traj_data(env, steps=sample_steps)]
        )
        self.obs_dim = self._gm.n_components
        
    def transform(self, obs: Any, action: Any):
        output = np.zeros(shape=self.obs_dim * self.num_actions)
        idx = self.obs_dim * action
        output[idx:idx+self.obs_dim] = self._gm.predict_proba([obs])[0]
        return output

    @property
    def output_shape(self) -> int:
        return self.obs_dim * self.num_actions

In [20]:
env = gym.make("MountainCar-v0", max_episode_steps=10000)
transform = GaussianMixObsOheActTransform(
    env, 
    params={"n_components": 4, "covariance_type": "tied"}
)
obs, _ = env.reset()
print(transform.transform(obs, 0))
print(transform.transform(obs, 1))
print(transform.transform(obs, 2))

[1.76822531e-04 6.77212111e-01 3.19142483e-01 3.46858365e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.76822531e-04 6.77212111e-01 3.19142483e-01 3.46858365e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.76822531e-04 6.77212111e-01 3.19142483e-01 3.46858365e-03]


### Random Binary Transform

In [21]:
class RndBinaryTransform(FeatTransform):
    def __init__(self, env: gym.Env, enc_size: int):
        if not isinstance(env.action_space, gym.spaces.Discrete):
            raise ValueError("env.action_space must be `spaces.Discrete`")              
        self.obs_space = env.observation_space
        if isinstance(self.obs_space, gym.spaces.Discrete):
            self.num_states = self.obs_space.n
        elif isinstance(self.obs_space, gym.spaces.Box):
            (self.obs_space.high - self.obs_space.low).item()
        else:
            raise ValueError("Unsupport observation_space. Must be Box[1d] or Discrete")
        self.num_actions = env.action_space.n
        self.enc_size = enc_size
        self._idx = [np.array(xs) for xs in 
            np.random.randint(
                0, 
                high=self.enc_size, 
                size=(self.num_states, math.floor(self.enc_size/2)),
            
            )
        ]
        
    def transform(self, obs: Any, action: Any):
        output = np.zeros(shape=self.enc_size * self.num_actions)
        idx = self.enc_size * action
        output[self._idx[obs]+idx] = 1
        return output

    @property
    def output_shape(self) -> int:
        return self.enc_size * self.num_actions

In [22]:
env = gym.make("CliffWalking-v0", max_episode_steps=10000)
transform = RndBinaryTransform(
    env, 
    enc_size=6
)
obs, _ = env.reset()
print(transform.transform(obs, 0))
print(transform.transform(obs, 1))
print(transform.transform(obs, 2))

[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Control with SARSA

In [23]:
def action_values(obs, actions: Sequence[int], weights, feat_transform: FeatTransform):
    state_action_m = np.zeros(shape=(len(actions), feat_transform.output_shape))
    for idx, action in enumerate(actions):
        state_action_m[idx, :] = feat_transform.transform(obs, action)
    return np.dot(state_action_m, weights), state_action_m

def semi_gradient_sarsa(env, alpha: float, gamma: float, epsilon: float, num_episodes: int, feat_transform: FeatTransform):
    actions = tuple(range(env.action_space.n))
    weights = np.zeros(feat_transform.output_shape, dtype=np.float64)
    returns = []
    
    for i in range(num_episodes):
        obs, _ = env.reset()
        state_qvalues, gradients = action_values(obs, actions, weights, feat_transform)
        rewards = 0
        # choose action
        if random.random() <= epsilon:
            action = env.action_space.sample()
        else:
            action = np.random.choice(np.flatnonzero(state_qvalues == state_qvalues.max()))

        while True:
            # greedy            
            next_obs, reward, term, trunc, _,  = env.step(action)
            rewards += reward
            
            if term or trunc:
                weights = weights + alpha * (reward - state_qvalues[action]) * gradients[action]
                break

            next_state_qvalues, next_gradients = action_values(next_obs, actions, weights, feat_transform)
            
            if random.random() <= epsilon:
                next_action = env.action_space.sample()
            else:
                # greedy
                next_action = np.random.choice(np.flatnonzero(next_state_qvalues == next_state_qvalues.max()))

            weights = weights + alpha * (
                reward + gamma * next_state_qvalues[next_action] - state_qvalues[action]
            ) * gradients[action]
            obs = next_obs
            action = next_action
            state_qvalues = next_state_qvalues
            gradients = next_gradients
        returns.append(rewards)
        if (i+1) % math.floor(num_episodes/5) == 0:
            print("Episode", i+1, "mean returns:", np.mean(returns))
    return weights


def play(env, weights, num_episodes: int, feat_transform):
    actions = tuple(range(env.action_space.n))
    returns = []
    for i in range(num_episodes):
        obs, _ = env.reset()
        rewards = 0
        while True:
            state_qvalues, _ = action_values(obs, actions, weights, feat_transform)
            action = np.random.choice(np.flatnonzero(state_qvalues == state_qvalues.max()))
            next_obs, reward, term, trunc, _,  = env.step(action)
            rewards += reward
            obs = next_obs
            if term or trunc:
                returns.append(rewards)
                break
    return np.mean(returns)

### Grid World

In [24]:
class GWObsWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gym.spaces.Box(
            high=np.array(
                    [env.observation_space["agent"][0].n, env.observation_space["agent"][1].n]
                ),
            low=np.zeros(shape=2, dtype=np.int32)
        )

    def observation(self, obs):
        return np.array(obs["agent"])


size, cliffs, exits, start = gridworld.parse_grid_from_text(
    ["oooooooooooo", "oooooooooooo", "oooooooooooo", "sxxxxxxxxxxg"]
)
env = GWObsWrapper(
    gridworld.GridWorld(size, cliffs, exits, start)
)
tile_transform = TileTransform(
    env=env,
    tiling_dim=8,
    hash_dim=512
)
weights = semi_gradient_sarsa(env, alpha=0.01, gamma=1.0, epsilon=0.1, num_episodes=2000, feat_transform=tile_transform)
play(gym.wrappers.TimeLimit(env, max_episode_steps=1000), weights, 20, feat_transform=tile_transform)

Num tilings 8 
 Flat dim: 2048
Episode 400 mean returns: -862.81
Episode 800 mean returns: -448.75625
Episode 1200 mean returns: -311.8675
Episode 1600 mean returns: -243.733125
Episode 2000 mean returns: -202.6185


-17.0

### RedGreen

In [25]:
class RGObsWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gym.spaces.Box(
            high=np.array(
                    [env.observation_space["pos"].n]
                ),
            low=np.array([0])
        )

    def observation(self, obs):
        return obs["pos"]

env = RGObsWrapper(
    redgreen.RedGreenSeq(["red", "green", "red", "green", "wait", "green"])
)

tile_transform = TileTransform(
    env=env,
    tiling_dim=4,
)

weights = semi_gradient_sarsa(env, alpha=0.01, gamma=1.0, epsilon=0.2, num_episodes=1000, feat_transform=tile_transform)
play(env, weights, 20, feat_transform=tile_transform)

Num tilings 4 
 Flat dim: 48
Episode 200 mean returns: -10.345
Episode 400 mean returns: -8.7025
Episode 600 mean returns: -8.121666666666666
Episode 800 mean returns: -7.8075
Episode 1000 mean returns: -7.661


-6.0

### Moutain Car

In [26]:
xs = np.array([1, 2, 3, 4])
xs_batch = np.reshape(np.tile(xs, reps=3), newshape=(-1, xs.size))
actions = np.array(range(3))
np.hstack((xs_batch, np.expand_dims(actions, -1)))

array([[1, 2, 3, 4, 0],
       [1, 2, 3, 4, 1],
       [1, 2, 3, 4, 2]])

In [27]:
env = gym.make("MountainCar-v0", max_episode_steps=10000)
tile_transform = TileTransform(
    env=env,
    tiling_dim=4,
)

weights = semi_gradient_sarsa(env, alpha=0.01, gamma=1.0, epsilon=0.2, num_episodes=1000, feat_transform=tile_transform)
play(gym.wrappers.TimeLimit(env, max_episode_steps=1000), weights, 20, feat_transform=tile_transform)

Num tilings 8 
 Flat dim: 384
Episode 200 mean returns: -230.93
Episode 400 mean returns: -194.2725
Episode 600 mean returns: -181.14
Episode 800 mean returns: -176.47375
Episode 1000 mean returns: -172.37


-132.75

### GEM

In [28]:
class StrictWeightedSumOfErrors(reward_functions.WeightedSumOfErrors):
    def __init__(
        self, reward_weights=None, normed_reward_weights=False, violation_reward=None
    ):
        super().__init__(
            reward_weights,
            normed_reward_weights,
            violation_reward,
            gamma=1.0,
            reward_power=1,
            bias=0,
        )

    def reward(self, state, reference, k=None, action=None, violation_degree=0.0):
        del k
        del action
        return (
            self._wse_reward(state, reference)
            + violation_degree * self._violation_reward
        )


In [29]:
class GEMObsWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self._mask = getattr(env.reference_generator, "referenced_states")
        state_obs_space, ref_state_obs_space = env.observation_space

        self._weights = getattr(env.reward_function, "_reward_weights")[self._mask]
        self._expo = getattr(env.reward_function, "_n")[self._mask]
        self._bias = getattr(env.reward_function, "_bias")
        self._denom = (state_obs_space.high - state_obs_space.low)[self._mask]
        self._prev_ref_state = None  # np.zeros_like(state_obs_space.high[self._mask])

        bounds = [
            np.abs(state_obs_space.high[self._mask] - ref_state_obs_space.low),
            np.abs(state_obs_space.high[self._mask] - ref_state_obs_space.high),
            np.abs(state_obs_space.low[self._mask] - ref_state_obs_space.high),
            np.abs(state_obs_space.low[self._mask] - ref_state_obs_space.low),
        ]
        obs_space_low = np.concatenate(
            [
                functools.reduce(np.minimum, bounds),
                # constraint violation
                np.array([0.0]),
            ]
        )
        obs_space_high = np.concatenate(
            [
                functools.reduce(np.maximum, bounds),
                # constraint violation
                np.array([1.0]),
            ]
        )
        self.observation_space = gym.spaces.Box(
            low=obs_space_low, high=obs_space_high, dtype=state_obs_space.dtype
        )
        self._cvfn = getattr(self.env.constraint_monitor, "check_constraints")

    def observation(self, observation):
        prev_ref_state = copy.copy(self._prev_ref_state)
        next_state, ref_state = observation
        cv = self._cvfn(next_state)
        next_state = next_state[self._mask]

        if prev_ref_state is None:
            prev_ref_state = ref_state

        wrapped_next_state = np.concatenate(
            [
                (abs(next_state - prev_ref_state) / self._denom) ** self._expo
                + self._bias,
                np.array([cv]),
            ]
        )
        self._prev_ref_state = ref_state
        return wrapped_next_state



In [30]:
rf = StrictWeightedSumOfErrors(
    violation_reward=-10,
)

env = GEMObsWrapper(
    gem.make("Finite-CC-PMSM-v0", reward_function=rf)
)

scale_transform = ScaleObsOheActTransform(
    env
)

weights = semi_gradient_sarsa(env, alpha=0.01, gamma=1.0, epsilon=0.2, num_episodes=1000, feat_transform=scale_transform)
play(env, weights, 20, feat_transform=scale_transform)

Episode 200 mean returns: -401.5997553584975
Episode 400 mean returns: -300.1405760688319
Episode 600 mean returns: -250.4832196093925
Episode 800 mean returns: -221.52691531155423
Episode 1000 mean returns: -204.85854552420057


-44.76319411204716

## Hyper-parameter search

In [31]:
def get_env_config(name: str):
    if name == "MountainCar-v0":
        env = gym.make(name, max_episode_steps=10000)
        params = {
            "encoding": tune.choice([
                {
                    "type": "tiles",
                    "args": {
                        "tiling_dim": tune.randint(2, 6),
                        
                    },
                },
                {
                    "type": "gaussian-mixture",
                    "args": {
                        "n_components": tune.randint(2, 20),
                        "covariance_type": tune.choice(["spherical", "tied", "diag", "full"]),
                    }
                },
                # {
                #     "type": "random-binary",
                #     "args": {
                #         "enc_size": tune.randint(6, 10)
                #     }
                # }
            ]),
            "learning": {
                "epsilon": tune.quniform(0.1, 0.3, 0.1),
                "alpha": tune.quniform(0.01, 0.08, 0.01)
            }
        }
        return env, params
    raise ValueError("Unknown env", name)



def create_feat_transform(env, name, args):
    # TODO: add hash_dim
    if name == "tiles":
        return TileTransform(env, **args)
    elif name == "gaussian-mixture":
        return GaussianMixObsOheActTransform(
            env,
            params=args
        )
    elif name == "random-binary":
        return RndBinaryTransform(env, **args)
    elif name == "scaled-obs":
        return ScaleObsOheActTransform(
            env
        )
    raise ValueError("Unknown transform %s", name)

    
def hps_control(env, config):
    alpha = config["learning"]["alpha"]
    epsilon = config["learning"]["epsilon"]

    # instantiate transform
    feat_transform = create_feat_transform(env, config["encoding"]["type"], config["encoding"]["args"])
    weights = semi_gradient_sarsa(
        env, alpha=alpha, gamma=1.0, epsilon=epsilon, num_episodes=1000, feat_transform=feat_transform
    )
    play(env, weights, 20, feat_transform=feat_transform)
    obs, _ = env.reset()
    return {
        "score": -1,
        "config": config,
        "obs": obs,
        }

env, param_space = get_env_config("MountainCar-v0")
tuner = tune.Tuner(
    functools.partial(hps_control, env), 
    param_space=param_space,
    tune_config=tune.TuneConfig(num_samples=10)
)
results = tuner.fit()
df = results.get_dataframe(filter_metric="score", filter_mode="max")

(func pid=21504) Num tilings 8 
(func pid=21504)  Flat dim: 600
(func pid=21497) Episode 200 mean returns: -181.94
(func pid=21502) Num tilings 8  [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(func pid=21502)  Flat dim: 216 [repeated 4x across cluster]
(func pid=21504) Episode 600 mean returns: -197.88166666666666 [repeated 8x across cluster]


2025-03-06 00:47:08,911	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/tiling_dim': <ray.tune.search.sample.Integer object at 0x3a2e1db10>}
2025-03-06 00:47:12,219	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/tiling_dim': <ray.tune.search.sample.Integer object at 0x16dc23cd0>}


(func pid=21504) Episode 1000 mean returns: -200.785 [repeated 7x across cluster]


2025-03-06 00:47:15,419	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/tiling_dim': <ray.tune.search.sample.Integer object at 0x3a2bd1ad0>}


(func pid=21502) Episode 800 mean returns: -419.51125 [repeated 5x across cluster]


...

2025-03-06 01:27:11,725	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/n_components': <ray.tune.search.sample.Integer object at 0x3a110ebd0>, 'encoding/args/covariance_type': <ray.tune.search.sample.Categorical object at 0x3a12d38d0>}
2025-03-06 01:27:31,627	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/n_components': <ray.tune.search.sample.Integer object at 0x3a2b7e0d0>, 'encoding/args/covariance_type': <ray.tune.search.sample.Categorical object at 0x3a30a9e90>}


(func pid=21499) Episode 600 mean returns: -9434.623333333333
(func pid=21501) Episode 600 mean returns: -9523.906666666666
(func pid=21505) Episode 800 mean returns: -9794.05375
(func pid=21499) Episode 800 mean returns: -9351.41
(func pid=21501) Episode 800 mean returns: -9474.79125
(func pid=21505) Episode 1000 mean returns: -9736.558


2025-03-06 01:46:03,675	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/n_components': <ray.tune.search.sample.Integer object at 0x3a3271a50>, 'encoding/args/covariance_type': <ray.tune.search.sample.Categorical object at 0x3a3270ed0>}


(func pid=21499) Episode 1000 mean returns: -9288.654
(func pid=21501) Episode 1000 mean returns: -9362.855


2025-03-06 01:54:23,683	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/n_components': <ray.tune.search.sample.Integer object at 0x16daf9ad0>, 'encoding/args/covariance_type': <ray.tune.search.sample.Categorical object at 0x16dafa010>}
2025-03-06 01:55:31,545	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'encoding/args/n_components': <ray.tune.search.sample.Integer object at 0x3a30cbd10>, 'encoding/args/covariance_type': <ray.tune.search.sample.Categorical object at 0x3a30ca610>}
2025-03-06 01:55:31,549	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency o